<a href="https://colab.research.google.com/github/lauragoon/Face-Mask-Detector/blob/main/svm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50

from sklearn import svm
from sklearn.model_selection import GridSearchCV

from skimage.io import imread
from skimage.transform import resize

In [3]:
def get_data(batch_size, directory):
  
  datagen = image.ImageDataGenerator()
  dta = datagen.flow_from_directory(directory, class_mode='binary', batch_size=batch_size)

  return dta

In [17]:
## generate data
train_path = 'drive/Shared drives/DSCI 303 Final Project/ImageDataset2/training'
test_path = 'drive/Shared drives/DSCI 303 Final Project/ImageDataset2/test'

train_data = get_data(1000, train_path)
test_data = get_data(500, test_path)

train_batch_images, train_batch_class = train_data.next()
test_batch_images, test_batch_class = test_data.next()

## pre-trained neural network to extract features
nn_model = ResNet50(input_shape=(256,256,3),
                    weights='imagenet',
                    include_top=False,
                    pooling='avg')

## feature extraction
img_features = nn_model.predict(train_batch_images).T

Found 6804 images belonging to 2 classes.
Found 2228 images belonging to 2 classes.


In [18]:
## pre-trained neural network to extract features
nn_model_test = ResNet50(input_shape=(256,256,3),
                    weights='imagenet',
                    include_top=False,
                    pooling='avg')

## feature extraction
test_img_features = nn_model_test.predict(test_batch_images).T

In [19]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid, verbose=1)
clf.fit(np.transpose(img_features), train_batch_class)

y_pred = clf.predict(np.transpose(test_img_features))


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   20.3s finished


In [20]:
num_data = len(y_pred)

accuracy = 0

for i in range(num_data):
    if y_pred[i] == test_batch_class[i]:
        accuracy += 1

print(accuracy / num_data)

0.998


In [ ]:
# reference: https://github.com/whimian/SVM-Image-Classification/blob/master/Image%20Classification%20using%20scikit-learn.ipynb